<a href="https://colab.research.google.com/github/amirsalhuv/MassDetector/blob/main/Classfication_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.2 Imports

In [54]:
# For image manipulation
import cv2
from PIL import Image
import numpy as np
import pandas as pd

# Image loading saving, copy ,measuring time
import time, json, os
import datetime
import pickle
# For plots
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm
import random

# Pytorch
import torch
from torch.utils.data import DataLoader,Dataset
from torch.nn import BCELoss
from torchvision.transforms import Compose, ToTensor, Normalize
from torch import nn,optim
import torch.nn.functional as F
from torchvision import models
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
from torchvision.models import vgg16_bn, VGG16_BN_Weights

# 1.3 Definitions


In [55]:
# Amir Salhuv:
dataset_dir = '/content/drive/MyDrive/Project_OpenU/CBIS-DDSM/'

# Yonatan Salhuv:
#dataset_dir = '/content/drive/MyDrive/'

data_folder = dataset_dir + 'Latest_model_data/'
patches_folder = data_folder + 'Patches/'
models_folder = dataset_dir + 'Full_model/Models/'
run_mode = "training_classifier_model"

# Classes and batch size
n_classes=2
batch_size = 16

# 3- Prepare the data for the model train
1. Cut the relevant patch according to the mask bounding boxes
2. store it locally (for as an .npy)


##3.1 Validation pathces

In [56]:
if run_mode == "cut_pathces":

  # Open the lookup table for classification
  lookup_table = pd.read_csv(dataset_dir+'Full_model/lookup_table.csv')

  # Define the minimal ROI
  min_roi_size = 100

  # Padding size
  pad = 200

  # Create an empty list for validation set
  X_val = list()
  y_val = list()

  # For visulazing the image and mask compared to origin, and lesion type
  x_origin = list()
  y_origin = list()
  val_lesion_type = []

  # Starting with validation images:
  val_image_dir = data_folder +'images/val'
  val_mask_dir = data_folder + 'masks/val'
  val_list = sorted(os.listdir(val_image_dir))

  # Steps:
  # 1. take an image
  # 2. Cut a window around the mask boundries
  # 3. append to whole list

  for i,image_path in tqdm(enumerate(val_list)):

    # step #1 - read the image and mask
    img = cv2.imread(os.path.join(val_image_dir, image_path))

    # Assesing the correct masks (0=benign,1=malignant)
    lesion_type_index = lookup_table.loc[lookup_table['image_id']== image_path]['pathology']
    if lesion_type_index.iloc[0] == 'MALIGNANT':
      lesion_type = 1
    else:
      lesion_type = 0

    mask = cv2.imread(os.path.join(val_mask_dir, image_path),cv2.IMREAD_GRAYSCALE)*255

    # Find contours in the binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # For each contour, find the bounding rectangle and create a new image
    for _, cnt in enumerate(contours):
      x, y, w, h = cv2.boundingRect(cnt)
      if w*h > min_roi_size:

        # add the origin image (so we can later view it)
        x_origin.append(image_path)
        y_origin.append(image_path)

        # lesion type
        val_lesion_type.append(lesion_type)

        # Add to train set
        X_val.append(cv2.resize(img[max(0,y-pad):min(mask.shape[0]-1,y+h+pad),max(0,x-pad):min(mask.shape[1]-1,x+w+pad),:],(256,256)))
        y_val.append(cv2.resize(mask[max(0,y-pad):min(mask.shape[0]-1,y+h+pad),max(0,x-pad):min(mask.shape[1]-1,x+w+pad)],(256,256)))

  # Convert and Save the data
  X_val = np.array(X_val)
  y_val = np.array(y_val)
  with open(data_folder + 'X_val_with_class.npy', 'wb') as f:
    np.save(f, X_val)
  with open(data_folder + 'y_val_with_class.npy', 'wb') as f:
    np.save(f, y_val)
  with open(data_folder + 'val_lesion_type.npy', 'wb') as f:
    np.save(f,val_lesion_type)


In [57]:
# Visulize one image
if run_mode == "cut_pathces":

  index = 100
  img = cv2.imread(os.path.join(val_image_dir, x_origin[index]))
  mask = cv2.imread(os.path.join(val_mask_dir, y_origin[index]))*255
  output = [X_val[index],y_val[index],img,mask]
  titles = ['patch','mask patch','original image','original mask']
  # Plot couple of examples
  fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10,10)) # Define a grid of 2 rows and 3 columns.

  for idx, ax in enumerate(axs.flatten()): # axs.flatten() gives us a 1D array to iterate over.
      ax.imshow(output[idx], cmap='gray')
      ax.set_title(titles[idx])
      ax.axis('off')

  plt.tight_layout()
  plt.show()

## 3.2 Training patches

In [58]:
if run_mode == "cut_pathces":

  # Define the minimal ROI
  min_roi_size = 100
  # Create an empty list for validation set
  X_train = list()
  y_train = list()

  # For visulazing the image and mask compared to origin
  x_origin = list()
  y_origin = list()
  train_lesion_type = []

  # Train images:
  train_image_dir = data_folder +'images/train'
  train_mask_dir = data_folder + 'masks/train'
  train_list = sorted(os.listdir(train_image_dir))

  # Steps:
  # 1. take an image
  # 2. Cut a window around the mask boundries
  # 3. append to whole list

  for i,image_path in tqdm(enumerate(train_list)):

    # step #1 - read the image and mask
    img = cv2.imread(os.path.join(train_image_dir, image_path))

    # Assesing the correct masks (0=benign,1=malignant)
    lesion_type_index = lookup_table.loc[lookup_table['image_id']== image_path]['pathology']
    if lesion_type_index.iloc[0] == 'MALIGNANT':
      lesion_type = 1
    else:
      lesion_type = 0

    # Read the mask
    mask = cv2.imread(os.path.join(train_mask_dir, image_path),cv2.IMREAD_GRAYSCALE)*255

    # Find contours in the binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # For each contour, find the bounding rectangle and create a new image
    for _, cnt in enumerate(contours):
      x, y, w, h = cv2.boundingRect(cnt)
      if w*h > min_roi_size:

        # add the origin image (so we can later view it)
        x_origin.append(image_path)
        y_origin.append(image_path)

        # lesion type
        train_lesion_type.append(lesion_type)

        # Add to train set
        X_train.append(cv2.resize(img[max(0,y-pad):min(mask.shape[0]-1,y+h+pad),max(0,x-pad):min(mask.shape[1]-1,x+w+pad),:],(256,256)))
        y_train.append(cv2.resize(mask[max(0,y-pad):min(mask.shape[0]-1,y+h+pad),max(0,x-pad):min(mask.shape[1]-1,x+w+pad)],(256,256)))

  # Convert and Save the data
  X_train = np.array(X_train)
  y_train = np.array(y_train)
  with open(data_folder + 'X_train_with_class.npy', 'wb') as f:
    np.save(f, X_train)
  with open(data_folder + 'y_train_with_class.npy', 'wb') as f:
    np.save(f, y_train)
  with open(data_folder + 'train_lesion_type.npy', 'wb') as f:
    np.save(f,train_lesion_type)


In [59]:
# Visulize one image
if run_mode == "cut_pathces":

  index = 100
  img = cv2.imread(os.path.join(train_image_dir, x_origin[index]))
  mask = cv2.imread(os.path.join(train_mask_dir, y_origin[index]))*255
  output = [X_train[index],y_train[index],img,mask]
  titles = ['patch','mask patch','original image','original mask']
  # Plot couple of examples
  fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10,10)) # Define a grid of 2 rows and 3 columns.

  for idx, ax in enumerate(axs.flatten()): # axs.flatten() gives us a 1D array to iterate over.
      ax.imshow(output[idx], cmap='gray')
      ax.set_title(titles[idx])
      ax.axis('off')

  plt.tight_layout()
  plt.show()

# 4 DataLoader

## 4.1 Define the dataloader

In [60]:
if run_mode == 'training_model' or run_mode == 'evaluate_model' or run_mode == 'training_classifier_model':

  #Assuming image/mask is from shape (256,256,C1/C2)
  class CustomDataset(Dataset):
      def __init__(self, images, masks,lesion_type, train=True):

          self.images = images
          self.masks = masks
          self.train = train
          self.lesion_type = lesion_type

        # In training, apply the following transformations
          self.train_image_transforms = transforms.Compose([

              transforms.Resize((224, 224)), # Resize to fit VGG16 input shape
              transforms.RandomHorizontalFlip(),
              transforms.RandomVerticalFlip(),
              transforms.RandomRotation(45),
              transforms.RandomResizedCrop(224),
              #transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
          ])

          # In training, apply the following transformations (including random transformation)
          self.train_mask_transforms = transforms.Compose([

              transforms.Resize((224, 224)), # Resize to fit VGG16 input shape
              transforms.RandomHorizontalFlip(),
              transforms.RandomVerticalFlip(),
              transforms.RandomRotation(45),
              #transforms.RandomResizedCrop(224),
              #transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
              transforms.ToTensor()
          ])

          # In training, apply the following transformations (Excluding random transformation)
          self.test_image_transforms = transforms.Compose([
              transforms.Resize((224, 224)),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
          ] )

          # In testing, apply the following transformations (Excluding random transformation)
          self.test_mask_transforms = transforms.Compose([
              transforms.Resize((224, 224)),
              transforms.ToTensor()
          ])

      def __len__(self):
          return len(self.images)

      def __getitem__(self, idx):

          # Get the image index
          #image = self.images[idx]*255
          image = self.images[idx]

          # Convert the numpy array to a PIL Image
          image = Image.fromarray(image.astype('uint8'))

          # Get the corresponding mask index (need to reshape to (256,256) to convert to PIL later
          #mask = self.masks[idx] *255
          mask = self.masks[idx]

          lesion_type = self.lesion_type[idx]

          # convert to fit to PIL image (256,256,1)-> (256,256)
          #mask = mask.squeeze()

          # convert to PIL image (not before updating from [0,1] to [0,255]), it will be normalized later
          mask = Image.fromarray(mask.astype('uint8'))

          if self.train:

              # make a seed with numpy generator
              seed = np.random.randint(2147483647)
              random.seed(seed)
              torch.manual_seed(seed)
              image = self.train_image_transforms(image)

              random.seed(seed)
              torch.manual_seed(seed)
              mask = self.train_mask_transforms(mask)

          else:
              image = self.test_image_transforms(image)
              mask = self.test_mask_transforms(mask)

          return image, mask, lesion_type


  # Load X and y from pickle files

  X_train = np.load(data_folder + 'X_train_with_class.npy')
  y_train = np.load(data_folder + 'y_train_with_class.npy')
  train_lesion_type = np.load(data_folder + 'train_lesion_type.npy')
  X_val = np.load(data_folder + 'X_val_with_class.npy')
  y_val = np.load(data_folder + 'y_val_with_class.npy')
  val_lesion_type = np.load(data_folder + 'val_lesion_type.npy')


  train_dataset = CustomDataset(X_train, y_train,train_lesion_type,train=True)
  val_dataset = CustomDataset(X_val, y_val,val_lesion_type, train=False)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## 4.2 Test the dataloader

In [61]:
if run_mode == 'testing_dataloader':

  index= 23
  # define the transform
  train_image_transforms = transforms.Compose([
    transforms.Resize((256, 256)), # Resize to fit VGG16 input shape
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # VGG16 normalization
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),

  ])
  X_train = np.load(data_folder + 'X_train_with_class.npy')
  y_train = np.load(data_folder + 'y_train_with_class.npy')
  train_lesion_type = np.load(data_folder + 'train_lesion_type.npy')
  # Load the image and convert to PIL image
  # Convert array to Image
  #image= X[0]*255
  image= X_train[index]
  img = Image.fromarray(image.astype('uint8'))
  img2= img

  # make a seed with numpy generator
  seed = np.random.randint(2147483647)
  random.seed(seed)
  torch.manual_seed(seed)
  img = train_image_transforms(img)

  # Assuming your DataLoader returns a batch of images and labels
  # we will display the first image from the batch

  def unnormalize(tensor):
      mean = torch.tensor([0.485, 0.456, 0.406])
      std = torch.tensor([0.229, 0.224, 0.225])
      for t, m, s in zip(tensor, mean, std):
          t.mul_(s).add_(m)
      return tensor

  img = unnormalize(img)
  img = transforms.ToPILImage()(img).convert("RGB")


  # In training, apply the following transformations (including random transformation)
  train_mask_transforms = transforms.Compose([

      transforms.Resize((256, 256)),
      transforms.ToTensor(),
      transforms.RandomHorizontalFlip(), # todo - return back
      transforms.RandomVerticalFlip(),
      transforms.RandomRotation(100),
  ])

  # Convert array to Image
  #mask1= y[0] *255
  mask1= y_train[index]
  mask1 = mask1.reshape((256,256))
  mask = Image.fromarray(mask1.astype('uint8'))
  random.seed(seed)
  torch.manual_seed(seed)
  mask = train_mask_transforms(mask)
  mask = transforms.ToPILImage()(mask).convert("RGB")


  # Create subplots
  fig, axs = plt.subplots(1, 4, figsize=(10, 5))  # adjust the size as needed

  # Show the images
  axs[0].imshow(img)
  axs[0].axis('off')  # to hide the axis
  axs[0].set_title('Image 1')

  axs[1].imshow(img2)
  axs[1].axis('off')  # to hide the axis
  axs[1].set_title('Image 2')

  axs[2].imshow(mask)
  axs[2].axis('off')  # to hide the axis
  axs[2].set_title('Mask')

  axs[3].imshow((np.array(mask))*img)
  axs[3].axis('off')  # to hide the axis
  axs[3].set_title('Image with Mask')

  plt.show()

  print(train_lesion_type[index])


## 4.2 Define stop and update learning rate conditions

In [62]:
if run_mode == 'training_model' or run_mode == 'training_classifier_model':
  NUM_OF_EPOCHS_TO_REDUCE_LR = 30
  NUM_OF_EPOCHS_TO_STOP = 50


  def model_checkpoint(model,valid_loss,best_val_loss,epoch,best_epoch,lr,last_lr_update,stop_training=False,model_option='patch'):
    if valid_loss < best_val_loss:

      # Imporvement in current validation loss, udpate best epoch
      print(f"Validation was improved, updated best loss: {valid_loss},best_epoch:{epoch},lr:{lr}\n")
      best_val_loss = valid_loss
      best_epoch = epoch
      last_lr_update = epoch

      # Save the model
      if model_option == 'patch':
        torch.save(model.state_dict(), models_folder + 'Patch_model_padding.pth')
      else:
        torch.save(model.state_dict(), models_folder + 'classifier_model_padding.pth')

    # Stop training if the validation loss doesn't improve for 10 epochs
    else:
      if epoch - best_epoch >= NUM_OF_EPOCHS_TO_STOP:
        print(f"No improvement in validation loss for {NUM_OF_EPOCHS_TO_STOP} epochs, stopping.")
        stop_training = True

      # If need only to reduce learning rate
      elif epoch - last_lr_update >= NUM_OF_EPOCHS_TO_REDUCE_LR:
        last_lr_update = epoch
        lr/=10
        print(f"No improvement in validation loss for {NUM_OF_EPOCHS_TO_REDUCE_LR} epochs, reducing learning rate to {lr}.\n")

    return stop_training,best_epoch,best_val_loss,lr,last_lr_update


# 5 Classification model training

## 5.1 Define the model

In [63]:
if run_mode == 'training_classifier_model':

  class Classifer_model(nn.Module):
      def __init__(self):
        super().__init__()

        # Load pre-trained VGG16 model and adjust the final layer
        self.classifer_model = models.vgg16(pretrained=True)

        #print(models.vgg16_bn(pretrained=True))

        # Change the final layer of VGG16 Model for Transfer Learning
        self.classifer_model.classifier[6] = torch.nn.Sequential(
            torch.nn.Linear(4096, 256),
            torch.nn.BatchNorm1d(256),  # Batch Normalization layer
            torch.nn.ReLU(),
            torch.nn.Dropout(0.4),
            torch.nn.Linear(256, 2), # Since we have two classes
            torch.nn.LogSoftmax(dim=1) # For using NLLLoss()
          )

      def forward(self, x):


        x = self.classifer_model(x)

        return x


## 5.2 Train

In [64]:
if run_mode == 'training_classifier_model':

  # Specify device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # Assume that we are on a CUDA machine, then this should print a CUDA device:
  print(f'Working on device={device}')

  # Initialize the patch model
  '''patch_model = VGGUnet(models.vgg16_bn, pretrained=True, out_channels=1)
  patch_model.load_state_dict(torch.load(models_folder + 'Patch_model_padding.pth'))
  patch_model.to(device)
  patch_model.eval()'''

  # Learning rate at start
  lr = 0.00001

  # choose a suitable value
  n_epochs = 1000

  # Define the model
  #classifer_model=  Classifer_model().to(device)

  classifer_model=  Classifer_model().to(device)
  #classifer_model= LesionClassifier().to(device)

  # Define the optimizer
  optimizer = optim.Adam(classifer_model.parameters(), lr=lr)

  # Define Loss Function and Optimizer
  criterion = torch.nn.NLLLoss()



Working on device=cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 259MB/s]


## 5.3  Metrics

In [65]:
def train_model(model, training_loader, valid_loader, n_epochs,lr,batch_size):

    # Init all values
    best_epoch = 1
    valid_loss = 0
    best_val_loss = float('inf')
    last_lr_update = best_epoch

    for epoch in range(n_epochs):
        classifer_model.train()

        train_loss = 0
        train_corrects = 0
        for images, masks,lesion_type in training_loader:

            images = images.to(device)
            masks = masks.to(device)
            lesion_type = lesion_type.to(device)

            output = classifer_model(images)
            loss = criterion(output,lesion_type)
            train_loss += loss.item() * images.size(0)


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(output, 1)
            train_corrects += torch.sum(preds == lesion_type.data)

        classifer_model.eval()
        valid_loss = 0
        val_corrects = 0
        with torch.no_grad():

            for images, masks, lesion_type in valid_loader:
                images = images.to(device)
                masks = masks.to(device)
                lesion_type = lesion_type.to(device)


                output = classifer_model(images)
                loss = criterion(output,lesion_type)
                valid_loss += loss.item() * images.size(0)

                _, preds = torch.max(output, 1)
                val_corrects += torch.sum(preds == lesion_type.data)

        train_loss = train_loss / len(y_train)
        valid_loss = valid_loss / len(y_val)

        train_acc = train_corrects.double() / len(y_train)
        valid_acc = val_corrects.double() / len(y_val)

        print(f'Epoch {epoch+1}/{n_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}, Training Acc: {train_acc}, Validation Acc: {valid_acc}')

        stop_training,best_epoch,best_val_loss,lr,last_lr_update = model_checkpoint(classifer_model,valid_loss,best_val_loss,epoch,best_epoch,lr,last_lr_update,model_option='Classifier')

        optimizer.param_groups[0]['lr'] = lr

        if stop_training:
          break

# Train the model
train_model(classifer_model, train_loader, val_loader, n_epochs,lr,batch_size)

Epoch 1/1000, Training Loss: 0.7685163025280236, Validation Loss: 0.669121546084892, Training Acc: 0.492379835873388, Validation Acc: 0.5821596244131455
Validation was improved, updated best loss: 0.669121546084892,best_epoch:0,lr:1e-05

Epoch 2/1000, Training Loss: 0.7588874988371435, Validation Loss: 0.6558456927398001, Training Acc: 0.5076201641266119, Validation Acc: 0.6338028169014085
Validation was improved, updated best loss: 0.6558456927398001,best_epoch:1,lr:1e-05

Epoch 3/1000, Training Loss: 0.7409832239989519, Validation Loss: 0.6730013081165547, Training Acc: 0.533411488862837, Validation Acc: 0.568075117370892
Epoch 4/1000, Training Loss: 0.7253154646190362, Validation Loss: 0.6368252850474326, Training Acc: 0.52989449003517, Validation Acc: 0.6572769953051644
Validation was improved, updated best loss: 0.6368252850474326,best_epoch:3,lr:1e-05

Epoch 5/1000, Training Loss: 0.6904013897581648, Validation Loss: 0.6336188086881324, Training Acc: 0.5873388042203985, Validatio